In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import copy
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
import sys
sys.path.append('../induction_motor_anomaly_detection/')
import modules,scaler


In [ ]:
normal_data = pd.read_csv('../data/combined_data.csv')
anomalous_data = pd.read_csv('../anomalous_data/anomalous_data.csv')

In [ ]:

train_data_features = modules.ElectricalFeatureExtractor(current_data=normal_data)
test_data_features = modules.ElectricalFeatureExtractor(current_data=anomalous_data)
train_scaler = scaler.Scaler()
train_scaler.fit_unlabelled_data(train_data_features.feature_dataframe[0])
train_scaled = train_scaler.transform(train_data_features.feature_dataframe[0])
test_scaled = train_scaler.transform(test_data_features.feature_dataframe[0])

In [ ]:
train_scaler = scaler.Scaler()
train_scaler.fit_unlabelled_data(normal_data)


In [ ]:
train_scaled = train_scaler.transform(normal_data)
anomalous_scaled = train_scaler.transform(anomalous_data)

In [ ]:
class LSTMAutoEncoder(Model):
    def __init__(self, activation='relu'):
        super(LSTMAutoEncoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.LSTM(64, activation=activation, return_sequences=True),
            tf.keras.layers.LSTM(32, activation=activation, return_sequences=True),
            tf.keras.layers.LSTM(16, activation=activation),
            tf.keras.layers.RepeatVector(3)])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.LSTM(16, activation=activation, return_sequences=True),
            tf.keras.layers.LSTM(32, activation=activation, return_sequences=True),
            tf.keras.layers.LSTM(64, activation=activation, return_sequences=True),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation="sigmoid"))])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
model = LSTMAutoEncoder()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=2,
                                                  mode='min')
model.compile(optimizer='adam', loss='mae')



In [ ]:
from tqdm import tqdm

file = 0
for i in tqdm(range(0, len(train_scaled), 10000)):
    history = model.fit(train_scaled[0:1000], train_scaled[0:1000],
                        epochs=50,
                        batch_size=128,
                        validation_data=(train_scaled[0:1000], train_scaled[0:1000]),
                        shuffle=True,
                        callbacks=[early_stopping],
                        verbose=0)
